### Importando bibliotecas

In [168]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm # teste z

### Carregando dataset

In [173]:
df = pd.read_csv('ab_data.csv', sep =';')

df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,1
1,804228,2017-01-12 08:01:45.159739,control,old_page,1
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,1
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,1
4,864975,2017-01-21 01:52:26.210827,treatment,new_page,1


In [174]:
df.shape

(294478, 5)

### Dicionário de dados

* **user_id**: Id do usuário da nossa amostra
* **timestamp**: Data/hora da coleta
* **group**: Identifica o grupo que cada usuário foi atribuído aleatóriamente
* **landing_page**: Identifica qual página do site o usuário acessou
* **converted**: Identifica se o usuário teve conversão de vendas ou não (0: Não e 1: Sim)

### Pergunta: Há alguma relação entre o novo e o antigo visual do site que influencie na Taxa de Conversão ?

Este é um problema clássico de teste de hipótese, ou teste A/B. O objetivo é entender se com o novo visual ou com novo layout do site, houve diferença significativa na taxa de conversão.

Identificamos porém, que nossos dados são categóricos, mesmo a coluna converted sendo visualmente numérica, ela representa uma categoria. Vamos tratar nossos dados e em seguida realizar o Teste de proporção ( Teste Z ).

### Tratamento de dados

In [175]:
# Removendo a coluna timestamp, pois não será necessária para o teste

df = df.drop('timestamp', axis = 1)

In [176]:
df.head()

,user_id,group,landing_page,converted
0,851104,control,old_page,1
1,804228,control,old_page,1
2,661590,treatment,new_page,1
3,853541,treatment,new_page,1
4,864975,treatment,new_page,1


#### Identificando se há valores nulos

In [177]:
df.isnull().sum()

user_id         0
group           0
landing_page    0
converted       0
dtype: int64

#### Identificando se há valores duplicados

Devemos remover valores duplicados em nosso dataset, pois isso fere uma das premissas, que é a premissa de **Independências**. Ela nos diz que, valores observados de uma categoria não devem estar relacionados ou dependentes dos valores observados em outra categoria.

In [178]:
df_duplicados = df.duplicated()
df_duplicados.value_counts()

False    294476
True          2
Name: count, dtype: int64

In [179]:
df_v2 = df.drop_duplicates()

In [180]:
df_v2.shape

(294476, 4)

### Definindo nossa Hipótese Nula e Hipótese Alternativa

#### Há alguma relação entre o novo e o antigo visual do site que influencie na Taxa de Conversão ?


**Hipóteses:**

* H0 ( Hipotese Nula ): Não há relação entre o novo e o antigo visual do site que influencie na taxa de conversão.
* H1 ( Hipótese Alternativa ): Há relação entre o novo e o antigo visual do site que influencie na taxa de conversão.

Levaremos em consideração um p-value de 0.05, ou seja:

* Caso p-value < 0.05 então rejeitamos H0, caso contrário falhamos em rejeitá-la

### Aplicando teste de hipótese

Aplicaremos o teste z

In [181]:
# Separando grupo de treatment
group_treatment = df_v2[(df_v2['group'] == 'treatment')]

# Separando grupo de control
group_control = df_v2[(df_v2['group'] == 'control')]

In [182]:
group_treatment.sample(5)

,user_id,group,landing_page,converted
193092,870309,treatment,new_page,1
217427,716977,treatment,new_page,0
249024,779329,treatment,new_page,1
42242,778707,treatment,new_page,0
8886,905086,treatment,new_page,0


In [183]:
group_control.sample(5)

,user_id,group,landing_page,converted
77568,651585,control,old_page,1
63586,920529,control,new_page,0
160786,897060,control,old_page,0
122532,801281,control,old_page,0
267158,781755,control,old_page,0


In [184]:
# Criando uma lista onde teremos a quantidade de conversão sucedidas em cada grupo

# grupo treatment e control com suas respectivas quantidades de converões
converted_success = [group_treatment['converted'].sum(), group_control['converted'].sum()]

# criando lista com o total de observações de cada grupo independente se teve conversões sucessidas ou não
total_observations = [len(group_treatment), len(group_control)]

In [185]:
converted_success

[18765, 17640]

In [186]:
total_observations

[147932, 146544]

In [187]:
z_test, p_value = sm.stats.proportions_ztest(converted_success, total_observations)

In [188]:
p_value

9.410978098521575e-08

### Calculando a proporção média para comparação

In [208]:
prop_treatment = df_v2[df_v2['group'] == 'treatment']['converted'].mean()

prop_control = df_v2[df_v2['group'] == 'control']['converted'].mean()

In [209]:
prop_treatment

0.1268488224319282

In [210]:
prop_control

0.12037340320995742

In [215]:
prop_treatment = df_v2[df_v2['group'] == 'treatment']['converted'].mean()

prop_control = df_v2[df_v2['group'] == 'control']['converted'].mean()

if p_value > 0.05:
    
    print('Não há evidências suficientes para rejeitar a H0.')
    print('Não há diferença estatisticamente significativa entre os grupos que influenciam na taxa de conversão.')

else:
    print('Há evidências suficientes para rejeitar a H0.')
    print('Há diferença estatisticamente significativa entre os grupos que influenciam na taxa de conversão.')
    
    if prop_treatment > prop_control:
        print('Grupo com maior influência na taxa de conversão: Treatment')
   
    elif prop_treatment < prop_control:
        print('Grupo com maior influência na taxa de conversão: Control')
   
    else:
        print('Não foi possível determinar qual grupo teve maior influência na taxa de conversão.')

Há evidências suficientes para rejeitar a H0.
Há diferença estatisticamente significativa entre os grupos que influenciam na taxa de conversão.
Grupo com maior influência na taxa de conversão: Treatment


### Conclusão

Com o teste realizado, identificamos com um que há uma diferença significativa entre os grupos **Treatment** e **Control** que influenciam na taxa de conversão.

Grupo de **Treatment** é o grupo de usuários que acessaram o website com novo layout, que também teve uma maior influência na taxa de conversão.